In [ ]:
import pandas as pd
from pymongo import MongoClient
import os

# Load CSV files
reviews_df = pd.read_csv('data/Books_rating.csv')
books_df = pd.read_csv('data/books_data.csv')

# Connect to MongoDB
client = MongoClient(os.getenv("MONGO_CONNECTION_STRING"))
db = client['BookRecommendationDB']

# Insert Raw Reviews Data
raw_reviews_collection = db['raw_reviews']
raw_reviews_data = reviews_df.to_dict(orient='records')
raw_reviews_collection.insert_many(raw_reviews_data)

# Insert Raw Books Data
raw_books_collection = db['raw_books']
raw_books_data = books_df.to_dict(orient='records')
raw_books_collection.insert_many(raw_books_data)

print("Raw data imported successfully!")
